In [ ]:
from evals.models.dino import DINO
from evals.models.stablediffusion import DIFT
from evals.models.clip import CLIP
from evals.models.sam import SAM
from evals.models.radio import RADIO

from PIL import Image
import numpy as np
import torch
from torchvision import transforms as transforms
from sklearn.decomposition import PCA
import torch
import matplotlib.pyplot as plt

import torch.nn.functional as F
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from sklearn.decomposition import PCA
import pdb
import random

In [ ]:
def PCA_visualize(feature, H, W, return_res=False, pcaed=False):

    feature_img_resized = F.interpolate(feature, 
                            size=(H, W), 
                            mode='bilinear', 
                            align_corners=True)
    feature_img_resized = feature_img_resized[0].permute(1, 2, 0)
    feature = feature_img_resized
    if feature.device != torch.device('cpu'):
        feature = feature.cpu()
    if not pcaed:
        pca = PCA(n_components=3)
        tmp_feature = feature.reshape(-1, feature.shape[-1]).detach().numpy()
        pca.fit(tmp_feature)
        pca_feature = pca.transform(tmp_feature)
        for i in range(3): # min_max scaling
            pca_feature[:, i] = (pca_feature[:, i] - pca_feature[:, i].min()) / (pca_feature[:, i].max() - pca_feature[:, i].min())
        pca_feature = pca_feature.reshape(feature.shape[0], feature.shape[1], 3)
    else:
        pca_feature = feature
    print(pca_feature.shape)
    if return_res:
        return pca_feature
    plt.imshow(pca_feature)  # cmap='gray'表示使用灰度颜色映射
    plt.axis('off')
    plt.show()
    
def load_image(url, transform_size):
    img = Image.open(url)
    img = np.array(img)[:, :, :3]
    H, W = img.shape[0], img.shape[1]
    img = Image.fromarray(img)
    rgb_transform = transforms.Compose(
                [
                    transforms.Resize((transform_size, transform_size)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )
    img = rgb_transform(img).to('cuda')
    img = img.unsqueeze(0).detach()
    return img, H, W

def transform_np_image_to_torch(image, transform_size):
    img = np.array(image)[:, :, :3]
    H, W = img.shape[0], img.shape[1]
    img = Image.fromarray(img)
    rgb_transform = transforms.Compose(
                [
                    transforms.Resize((transform_size, transform_size)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )
    img = rgb_transform(img).to('cuda')
    img = img.unsqueeze(0).detach()
    return img, H, W

def get_point_cloud(obs):
    camera_params = obs["camera_param"]
    images = obs["image"]
    camera_dicts = camera_params
    for camera_name in camera_dicts:
        camera_intrinsic = camera_dicts[camera_name]["intrinsic_cv"]
        cam2world_matrix = camera_dicts[camera_name]["cam2world_gl"]
        Rtilt_rot = cam2world_matrix[:3, :3] @ np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
        Rtilt_trl = cam2world_matrix[:3, 3]
        cam2_wolrd = np.eye(4)
        cam2_wolrd[:3, :3] = Rtilt_rot
        cam2_wolrd[:3, 3] = Rtilt_trl
        camera_dicts[camera_name]["cam2world"] = cam2_wolrd
        camera_image = images[camera_name]
        camera_rgb = camera_image["rgb"]
        camera_depth = camera_image["depth"]
        point_cloud_world, per_point_rgb, = tanslation_point_cloud(camera_depth, camera_rgb,
                                                                                    camera_intrinsic, cam2_wolrd)
            
        camera_dicts[camera_name]["point_cloud_world"] = point_cloud_world
        camera_dicts[camera_name]["per_point_rgb"] = per_point_rgb
        camera_dicts[camera_name]["rgb"] = camera_rgb
        camera_dicts[camera_name]["depth"] = camera_depth
        camera_dicts[camera_name]["camera_intrinsic"] = camera_intrinsic
        # view_point_cloud_parts(point_cloud=point_cloud_world, mask=seg_mask)
        # view_point_cloud_parts(point_cloud=point_cloud_world, rgb=per_point_rgb)
    return camera_dicts

def tanslation_point_cloud(depth_map, rgb_image, camera_intrinsic, cam2world_matrix):
    depth_map = depth_map.reshape(depth_map.shape[0], depth_map.shape[1])
    rows, cols = depth_map.shape[0], depth_map.shape[1]
    
    u, v = np.meshgrid(np.arange(cols), np.arange(rows))
    z = depth_map
    x = (u - camera_intrinsic[0][2]) * z / camera_intrinsic[0][0]
    y = (v - camera_intrinsic[1][2]) * z / camera_intrinsic[1][1]
    points = np.dstack((x, y, z))
    per_point_xyz = points.reshape(-1, 3)
    per_point_rgb = rgb_image.reshape(-1, 3)
    # view_point_cloud_parts(per_point_xyz, actor_seg)
    point_xyz = [per_point_xyz]
    point_rgb = [per_point_rgb]
    # print('!', point_xyz[0].shape, point_rgb[0].shape)

    if len(point_xyz) > 0:
        pcd_camera = np.concatenate(point_xyz)
        point_rgb = np.concatenate(point_rgb)
        pcd_world = pc_camera_to_world(pcd_camera, cam2world_matrix)
        return pcd_world, point_rgb
    else:
        return None, None

def pc_camera_to_world(pc, extrinsic):
    R = extrinsic[:3, :3]
    T = extrinsic[:3, 3]
    pc = (R @ pc.T).T + T
    return pc

# Open3D

In [ ]:
with open('./1.pickle', 'rb') as file:
    data = pickle.load(file)
camera_list = list(data['image'].keys())

### CLIP

In [ ]:
model = CLIP(layer=11).to('cuda')
len(model.visual.transformer.resblocks)

### SAM

In [ ]:
model = SAM(arch="vit_h", device='cuda')
len(model.vit.blocks)

In [ ]:
rgb_list = []
feature = []
with torch.no_grad():
    for camera in camera_list:
        image = data['image'][camera]['rgb']
        rgb_list.append(image)
        img, H, W = transform_np_image_to_torch(image, transform_size=1600)
        res = model(img)
        feature.append(res.cpu())

    feature = np.array(feature)
    

In [ ]:
new_order = (0, 1, 3, 4, 2)
new_feature = np.transpose(feature, new_order)
orig_shape = new_feature.shape
new_feature = new_feature.reshape(-1, new_feature.shape[-1])
print(f'orig: {feature.shape}, new: {new_feature.shape}')

pca = PCA(n_components=3)
pca.fit(new_feature)
pca_features = pca.transform(new_feature)

for i in range(3):
    # min_max scaling
    pca_features[:, i] = (pca_features[:, i] - pca_features[:, i].min()) / (pca_features[:, i].max() - pca_features[:, i].min())

tmp = orig_shape[:-1] + (3,)
feature_rgb = pca_features.reshape(tmp)
feature_rgb = np.transpose(feature_rgb, (0, 1, 4, 2, 3))
print(feature_rgb.shape)

In [ ]:
for i in range(len(camera_list)):
    camera = camera_list[i]
    image = data['image'][camera]['rgb']
    H, W = image.shape[:2]
    rgb = torch.Tensor(feature_rgb[i])
    res = PCA_visualize(rgb, H, W, return_res=True, pcaed=True)
    data['image'][camera]['rgb'] = res
    plt.subplot(3, 2, i+1)
    plt.title(camera)
    plt.imshow(res)
    plt.axis('off')

In [ ]:
pcd_dict = get_point_cloud(data)

In [ ]:
pcd = np.empty((0, 3), dtype=np.float16)
pcd_rgb = np.empty((0, 3), dtype=np.float16)
for camera in camera_list:
    tmp_xyz = pcd_dict[camera]['point_cloud_world']
    tmp_rgb = pcd_dict[camera]['per_point_rgb']
                               
    t = 0.1 # crop ground
    # crop ground
    bool_mask = tmp_xyz[:, 2] > t
    masked_idx = np.where(bool_mask)[0]
    masked_xyz = tmp_xyz[masked_idx]
    masked_rgb = tmp_rgb[masked_idx]

    pcd = np.concatenate((pcd, masked_xyz), axis=0)   
    pcd_rgb = np.concatenate((pcd_rgb, masked_rgb), axis=0) 
print(pcd.shape)

In [ ]:
pcd = pcd
idx = random.sample(range(0, pcd.shape[0]), 100000)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pcd[idx, 0], pcd[idx, 1], pcd[idx, 2], c=pcd_rgb[idx], s=1, marker='o')
# 设置图表标题
ax.set_title('3D Point Cloud Rendering')
# 显示图表
plt.show()